In [ ]:
#辞書の展開関数
def normalize_dic(dic):
    ret = {}
    for key in dic.keys():
        if type(dic[key])==list:
            for i in range(len(dic[key])):
                if type(dic[key][i]) != list and type(dic[key][i])!=dict:
                    ret["%s_%i" % (key, i)] = dic[key][i]
                else:
                    nd = normalize_dic(dic[key][i])
                    if "name" in nd.keys():
                        for key2 in nd.keys():
                            if key2 == "name":
                                continue
                            ret["%s_%s_%s" % (key, nd['name'], key2)] = nd[key2]
                    else:
                        for key2 in nd.keys():
                            ret["%s_%i_%s" % (key, i, key2)] = nd[key2]
        elif type(dic[key])==dict:
            nd = normalize_dic(dic[key])
            for key2 in nd.keys():
                ret["%s_%s" % (key, key2)] = nd[key2]
        else:
            ret[key] = dic[key]
    return ret

def longestSubstringFinder(string1, string2):
    answer = ""
    len1, len2 = len(string1), len(string2)
    if len1 > len2:
        len_ = len2
    else:
        len_ = len1
    for i in range(len_):
        if string1[i] == string2[i]:
            answer+= string2[i]
    return answer

def key_filter(dic, key, isShortenKey=False):
    dk = []
    for k in dic:
        if key not in k:
            dk.append(k)
    for k in dk:
        dic.pop(k)
    if isShortenKey:
        ret_dic = {}
        keys = list(dic.keys())
        m = keys[0]
        for k in keys:
            m = longestSubstringFinder(k, m)
        for k in keys:
            nk = k.replace(m, "")
            ret_dic[nk] = dic[k]
        dic = ret_dic
    return dic

In [7]:
import glob
import os
import pandas as pd
import json
from datetime import datetime, timedelta, timezone

vi_dir = "/data/output-directory_all/physical-infrastructure-bgpnw2/"

vi_json_list = glob.glob(os.path.join(vi_dir, "*.json"))#get file name
vi_json_list.sort()

vi_dic_list = []

for vi_json in vi_json_list: #jsonの読み出し
    with open(vi_json) as f:
        vi_dic = json.load(f)
        vi_dic_list.append(vi_dic)

normalized_vi_dic = []
df = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)

starttime = vi_dic_list[0]['time']#データのはじめの時間
time = starttime

for vi_dic_temp in vi_dic_list:
    normalized_vi_dic = normalize_dic(vi_dic_temp)
    
    if normalized_vi_dic['time'] == time:#時刻通りにファイルが存在するとき
        normalized_vi_dic['time'] = datetime.fromtimestamp(normalized_vi_dic['time']) #unixtime to DateTime
        df_temp = pd.DataFrame([normalized_vi_dic])
        df = df.append(df_temp)
        time = time + 60
        
    else:#欠落ファイルがある時刻に空の行を挿入
        while time != normalized_vi_dic['time']:
            df_temp = pd.DataFrame([normalized_vi_dic])#とりあえず前の時刻の辞書を読み込んで，同じ型のデータフレームをつくる
            df_temp = df_temp.where(df_temp.isnull(), None) #辞書を展開したdataframeの値を全部Noneに置き換え
            df_temp.at[df.index[0], 'time'] = datetime.fromtimestamp(time)#タイムスタンプを追加
            df = df.append(df_temp)
            time = time + 60  
            
        normalized_vi_dic['time'] = datetime.fromtimestamp(normalized_vi_dic['time']) #unixtime to DateTime
        df_temp = pd.DataFrame([normalized_vi_dic])
        df = df.append(df_temp)
        time = time + 60
        
df.to_pickle('./physical-infrastructure-bgpnw2.pkl')
df